In [14]:
from app import db
from app.models import User, Post, Comment, PostComment, ParentChild

In [15]:
def get_path(comment_id, path=None):
    ''' Gets the comment tree path '''
    comment = Comment.query.filter_by(id=comment_id).first()  # Create comment object
    if path is None:  # Create path list on first pass
        path = []
        
    if comment.id not in path:  # Start the path with self ID
        path.append(comment.id)
        
    if comment.parent_id:  # Append parent ID to the path
        path.append(comment.parent_id)
        return get_path(comment.parent_id, path=path)  # Restart loop with updated path
    
    else:
        path.reverse()
        return path

In [16]:
def get_comment_tree_list(post_id):
    ''' Returns a list of comment tree paths '''
    comments = Comment.query.filter_by(post_id=post_id).all()
    comment_tree_list = list()
    for comment in comments:
        comment_path = get_path(comment.id)
        comment_tree_list.append(comment_path)
    return comment_tree_list

In [17]:
def check_sublist(sublist, full_list):
    check = False

    for item in full_list:
        if set(sublist).issubset(set(item)):
            check = True
            return check
        else:
            pass

    return check

def get_pruned_list(input_tree_list, pruned_list=None):
        """ Removes redundant paths from list of comment tree paths """
        tree_list = input_tree_list.copy()
        if pruned_list == None:
            pruned_list = list()
        if len(tree_list) > 0:
            for tree in tree_list:
                tree_list.remove(tree)
                if not check_sublist(tree, tree_list):
                    pruned_list.append(tree)
                    return get_pruned_list(tree_list, pruned_list=pruned_list)
                else:
                    return get_pruned_list(tree_list, pruned_list=pruned_list)
        else:
            return pruned_list

In [18]:
ctl = get_comment_tree_list(1)
print(ctl)

[[1], [1, 2], [1, 3], [1, 4], [1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 3, 8], [1, 2, 9], [10]]


In [19]:
pct = get_pruned_list(ctl)
print(pct)

[[1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 3, 8], [1, 2, 9], [10]]
